# Domherren
## Datenimport für Wiagweb 1

Quelle: `W:\Domstifte GS\GS_Domherren_DB.accdb`

- [Literatur](#Literatur)
- [Personendaten](#Personendaten)
- [Literaturangaben](#Literaturangaben)
- [Ämter](#Ämter)
- [Klöster](#Klöster)
- [Klöster (Dump)](#Klöster-(Dump))
- [Normierte Ämter](#Normierte-Ämter)
- Hilfstabellen
  - [`cn_online`](#cn_online)
  - [`cn_officelookup`](#cn_officelookup)
  - [`cn_office.location_show`](#cn_office.location_show)
  - [`cn_era`](#cn_era)
  - [`cn_namelookup`](#cn_namelookup)
  - [`cn_idlookup`](#cn_idlookup)
  
  
2021-11-11, 2021-12-16, 2022-01-11 (GS: neue Hilftabellen),  
2022-02-18 (Print)

In [1]:
# wds_path="C:\\Users\\Georg\\Documents\\projekte\\WiagDataSetup.jl"
wds_path="..\\.."

"..\\.."

In [2]:
cd(wds_path)

In [3]:
pwd()

"C:\\Users\\georg\\Documents\\projekte\\WiagDataSetup.jl"

In [4]:
using Pkg

In [5]:
Pkg.activate(".")

In [6]:
using Revise, MySQL

In [7]:
using WiagDataSetup

In [8]:
using DataFrames

In [9]:
Wds = WiagDataSetup

WiagDataSetup

In [10]:
Wds.setDBWIAG(user="georg", db="wiag_in")

Passwort für User georg: ········


MySQL.Connection(host="127.0.0.1", user="georg", port="3306", db="wiag_in")

In [11]:
db_exec(sql) = DBInterface.execute(Wds.dbwiag, sql) |> DataFrame; # abgelöst durch Wds.sql_df

Datenbank anlegen: `wiag_in`  
Tabellen anlegen: siehe `domherren_create.sql`
- `cn_canon`
- `cn_office`
- `cn_canon_reference`
- `cn_reference`

## Literatur

In [12]:
sql = "SELECT * FROM domherr.tbl_reference"
df_vol = Wds.sql_df(sql);

In [13]:
names(df_vol)

5-element Vector{String}:
 "ID_Ref"
 "Literatur"
 "Verfasser"
 "OnlineRessource"
 "Kurztitel"

In [13]:
columns = [
    :ID_Ref => :id,
    :Literatur => :title,
    :Verfasser => :author,
    :OnlineRessource => :online_resource,
    :Kurztitel => :shorttitle,
]

5-element Vector{Pair{Symbol, Symbol}}:
          :ID_Ref => :id
       :Literatur => :title
       :Verfasser => :author
 :OnlineRessource => :online_resource
       :Kurztitel => :shorttitle

In [14]:
table_name = "cn_reference"
Wds.filltable!(table_name, select(df_vol, columns), clear_table = true)

┌ Info: Rows inserted: 48
└ @ WiagDataSetup C:\Users\georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:1202


48

## Personendaten

Der Status kann 'online' oder 'merged' sein, damit Literaturangaben gesammelt werden können.

In [18]:
sql = "SELECT ID_Domherr as id, " *
"Eintrag_Quelle as item_reference, Seite_Quelle as page_reference, Quelle as id_reference, ID_Quelle as id_in_reference, " *
"Vorname as givenname, Praefix as prefix_name, Familienname as familyname, " *
"Sterbedatum as date_death, Geburtsdatum as date_birth, " *
"Orden as religious_order, " *
"URL_Wikipedia as wikipedia_url, " *
"fertig as isready, " *
"Bemerkung_Red as annotation_ed, " *
"Bistum as diocese, " *
"AkademischerTitel as academic_title, " *
"GND_ID as gnd_id, " *
"GSN_ID as gsn_id, " *
"VIAF_id as viaf_id," *
"Wikidata_ID as wikidata_id, " *
"Autor_Original as author_orig, " *
"Familienname_Variante as familyname_variant, Vorname_Variante as givenname_variant, " *
"Kommentar_Name as comment_name, Kommentar_Person as comment_person, " *
"BelegdatumBeginn as date_min_in, BelegdatumEnde as date_max_in, " *
"WIAG_ID_Bischof as wiag_episc_id, " *
"Identisch_mit as same_as, Merged_Into as merged_into, " *
"Status as status " *
"FROM domherr.tbl_domherren " *
"WHERE Status = 'online' OR Status = 'merged'"
df_dh = Wds.sql_df(sql);

*2022-02-17* Wir haben ein Problem mit den Daten in Access. Der Export in MySQL scheitert für `tbl_domherren`. Lies die Daten über CSV ein.

In [15]:
using CSV

In [16]:
filename = "C:\\Users\\georg\\Documents\\projekte-doc\\WIAGweb\\data\\tbl_Domherren.csv"
df_dh = CSV.read(filename, DataFrame);

In [17]:
columns = [
    :ID_Domherr => :id,
    :Eintrag_Quelle => :item_reference,
    :Seite_Quelle => :page_reference,
    :Quelle => :id_reference,
    :ID_Quelle => :id_in_reference,
    :Vorname => :givenname,
    :Praefix => :prefix_name,
    :Familienname => :familyname,
    :Sterbedatum => :date_death,
    :Geburtsdatum => :date_birth,
    :Orden => :religious_order, 
    :URL_Wikipedia => :wikipedia_url, 
    :fertig => :isready, 
    :Bemerkung_Red => :annotation_ed, 
    :Bistum => :diocese, 
    :AkademischerTitel => :academic_title, 
    :GND_ID => :gnd_id, 
    :GSN_ID => :gsn_id, 
    :VIAF_ID => :viaf_id,
    :Wikidata_ID => :wikidata_id, 
    :Autor_Original => :author_orig, 
    :Familienname_Variante => :familyname_variant, 
    :Vorname_Variante => :givenname_variant, 
    :Kommentar_Name => :comment_name, 
    :Kommentar_Person => :comment_person, 
    :BelegdatumBeginn => :date_min_in, 
    :BelegdatumEnde => :date_max_in, 
    :WIAG_ID_Bischof => :wiag_episc_id, 
    :Identisch_mit => :same_as, 
    :Merged_Into => :merged_into, 
    :Status => :status,
];

In [18]:
select!(df_dh, columns);

In [19]:
Wds.clean_up!(df_dh)

In [20]:
has_proper_status(status) = isequal(status, "online") || isequal(status, "merged")

has_proper_status (generic function with 1 method)

In [21]:
subset!(df_dh, :status => ByRow(has_proper_status));

In [22]:
size(df_dh)

(3420, 31)

Info 2021-12-16: 3 Neuzugänge

In [14]:
df_r = filter(:id => isequal(10949), df_dh, view = true)
df_r[1, [:id, :status, :givenname, :familyname_variant]]

,id,status,givenname,familyname_variant
,Int32,SubStri…,SubStri…,SubStri…?
88,10949,online,Wilhelm Egon,Fürstenberg-Heiligenberg; Reichsgraf von Fürstenberg-Heiligenberg; Fürst von Fürstenberg-Heiligenberg


Das wird von der Funktion `fillcnnamelookup` abgefangen.

Feld `isready` in Int umwandeln (entfällt, wenn die Daten aus einer CSV-Datei stammen)

In [51]:
isready(bit) = ismissing(bit) ? 0 : (bit == MySQL.API.Bit(1) ? 1 : 0)
transform!(df_dh, :isready => ByRow(isready) => :isready);

In [23]:
filter(:isready => isequal(1), df_dh[1:5, [:id, :givenname, :isready]])

,id,givenname,isready
,Int64,SubStri…?,Int64
1,10007,Adolf Damian Wolfgang,1
2,10014,Philipp Wilderich,1
3,10084,Franz Wilhelm,1
4,10091,Johann Friedrich Arnold,1


## Literaturangaben

direkte Einträge (alle Einträge haben einen Literaturverweis)

In [23]:
ref1(status, merged_into) = isequal(status, "online") && ismissing(merged_into)
df_dh_ref1 = filter([:status, :merged_into] => ref1, df_dh);

In [25]:
size(df_dh_ref1)

(2551, 31)

In [24]:
columns = [
    :id => :id_canon,
    :id => :id_canon_orig,
    :id_reference => :id_reference,
    :item_reference => :item_reference,
    :page_reference => :page_reference,
    :id_in_reference => :id_in_reference,
];

In [25]:
table_name = "cn_canon_reference"
Wds.filltable!(table_name, select(df_dh_ref1, columns), clear_table = true)

┌ Info: Rows inserted: 2586
└ @ WiagDataSetup C:\Users\georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:1202


2586

In [26]:
ref2(status, merged_into) = isequal(status, "merged") && !ismissing(merged_into)
df_dh_ref2 = filter([:status, :merged_into] => ref2, df_dh);

In [27]:
size(df_dh_ref2)

(832, 31)

In [28]:
columns_2nd = [
    :merged_into => :id_canon,
    :id => :id_canon_orig,
    :id_reference => :id_reference,
    :item_reference => :item_reference,
    :page_reference => :page_reference,
    :id_in_reference => :id_in_reference,
];

In [29]:
table_name = "cn_canon_reference"
Wds.filltable!(table_name, select(df_dh_ref2, columns_2nd), clear_table = false)

┌ Info: Rows inserted: 832
└ @ WiagDataSetup C:\Users\georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:1202


832

Auszählung zu häufigsten Literaturverweisen
```
+----------+---+
| id_canon | n |
+----------+---+
|    14077 | 5 |
|    18785 | 5 |
|    18727 | 4 |
|    19313 | 4 |
|    18702 | 4 |
|    18677 | 4 |
|    11041 | 4 |
|    19318 | 4 |
|    18770 | 4 |
|    18783 | 4 |
|    11075 | 4 |
|    11007 | 3 |
+----------+---+
```

## Lebensdaten
Hier werden nur noch Einträge benötigt mit Status = 'online'

In [30]:
df_dh_online = filter(:status => isequal("online"), df_dh);

In [31]:
nrow(df_dh), nrow(df_dh_online)

(3420, 2587)

`date_hist_first` ist befüllt, allerdings mit Text.  
Jahreszahlen auslesen.

In [32]:
date_hist_first(date) = Wds.parsemaybe(ismissing(date) ? date : String(date), :lower)
transform!(df_dh_online, :date_min_in => (x -> date_hist_first.(x)) => :date_hist_first);

In [33]:
date_hist_last(date) = Wds.parsemaybe(ismissing(date) ? date : String(date), :upper)
transform!(df_dh_online, :date_max_in => (x -> date_hist_last.(x)) => :date_hist_last);

In [34]:
numdate_death(date) = Wds.parsemaybe(ismissing(date) ? date : String(date), :upper)
transform!(df_dh, :date_death => (x -> numdate_death.(x)) => :numdate_death);

In [35]:
numdate_birth(date) = Wds.parsemaybe(ismissing(date) ? date : String(date), :lower)
transform!(df_dh_online, :date_birth => (x -> numdate_birth.(x)) => :numdate_birth);

Die Einträge für Wikipedia-Seiten enthalten zum Teil '#' und sind doppelt.

In [36]:
transform!(df_dh_online, :wikipedia_url => (x -> Wds.fix_Wikipedia_URL.(x)) => :wikipedia_url);

Info

In [73]:
df_info = filter(:wikipedia_url => !ismissing, select(df_dh_online, [:id, :wikipedia_url]));

In [74]:
df_info[1:7, :]

,id,wikipedia_url
,Int64,SubStri…?
1,10867,Franz_Karl_von_Landsberg_zu_Erwitte
2,10881,Johann_Ernst_von_L%C3%B6wenstein-Wertheim-Rochefort
3,10885,Clemens_August_von_Ascheberg_zu_Venne
4,10886,Karl_Ludwig_von_Ascheberg_zu_Venne
5,10887,Franz_Arnold_von_der_Asseburg
6,10888,Joseph_Clemens_von_Bayern
7,10891,Franz_C%C3%B6lestin_von_Beroldingen


In [37]:
table_name = "cn_canon"
Wds.filltable!(table_name, select(df_dh_online, Not([:date_min_in, :date_max_in])), clear_table = true)

┌ Info: Rows inserted: 2587
└ @ WiagDataSetup C:\Users\georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:1202


2587

## Ämter

Betrachte nur Ämter von relevanten Domherren (`status = 'online'`). Verwende die neu beschriebene Tabelle `cn_canon`.

In [38]:
sql = "SELECT ID_Amt as id_in_dh, ID_Domherr as id_canon, ID_Fremdschlüssel as id_in_reference, " *
"ID_Amt_norm as id_office_norm, Amtsart as office_name, " *
"Amtsbeginn as date_start, Amtsende as date_end, " *
"Bistum as diocese, Institution as institution, ID_Kloster as id_monastery, " *
"Profession as profession, Amt_weltlich as office_mundane, Herrschaftsgebiet as dominion, " *
"Ort as location, Archidiakonat as archdeacon_territory, " *
"Dignität as dignity, " *
"Amt_Kommentar as comment, " *
"Sort as sortkey " *
"FROM domherr.tbl_domherren_aemter " *
"WHERE ID_Domherr in (SELECT id FROM wiag_in.cn_canon WHERE status = 'online')"
df_oc = Wds.sql_df(sql);

In [39]:
Wds.clean_up!(df_oc);

In [40]:
size(df_oc)

(5679, 18)

In [41]:
date_start(date) = Wds.parsemaybe(date, :lower)
transform!(df_oc, :date_start => (x -> date_start.(x)) => :numdate_start);

┌ Warning: Could not parse 
│   s = unbekannt
└ @ WiagDataSetup C:\Users\georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:1111
┌ Warning: Could not parse 
│   s = unbekannt
└ @ WiagDataSetup C:\Users\georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:1111
┌ Warning: Could not parse 
│   s = wohl im 9. Jahrhundert
└ @ WiagDataSetup C:\Users\georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:1111


In [42]:
date_end(date) = Wds.parsemaybe(date, :upper)
transform!(df_oc, :date_end => (x -> date_end.(x)) => :numdate_end);

*2022-02-24* Befülle `time_sortkey` erst aus `date_end` dann aus `date_start`

In [46]:
df_oc_corr = subset(df_oc, :date_end => ByRow(isequal("(vor) 1464")), view = true)

,id_in_dh,id_canon,id_in_reference,id_office_norm,office_name,date_start,date_end
,Int32,Int32,String?,Int32,String?,String?,String?
1,36568,13672,66,19,Domherr,vor 1464,(vor) 1464


In [106]:
df_oc[!, :time_sortkey] .= missing;

In [107]:
transform!(df_oc, :date_end => ByRow(Wds.parse_year_sort) => :time_sortkey);

┌ Warning: could not parse 
│   s = (vor) 1464
└ @ WiagDataSetup C:\Users\georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:1495


In [177]:
function update_col(from, to)
    cand = Wds.parse_year_sort(from)
    if ismissing(to) || cand < to
        return cand
    else
        return to
    end
end

update_col (generic function with 1 method)

In [178]:
transform!(df_oc, [:date_start, :time_sortkey] => ByRow(update_col) => :time_sortkey);

Welche Diözesen sind in der Datenbank?  
befülle `diocese_in_db`

In [49]:
sql = "SELECT diocese FROM wiag.diocese"
df_dioc = Wds.sql_df(sql);

In [50]:
in_db(diocese) = ismissing(diocese) ? 0 : (in(diocese, df_dioc.diocese) ? 1 : 0)
df_oc[!, :diocese_in_db] .= 0;
for row in eachrow(df_oc)
    row[:diocese_in_db] = in_db(row[:diocese])
end

*2022-02-18*: Die Klosterdatenbank ist aktuell und komplett in WIAG übernommen.  
Die Folgenden Schritte sind aktuell nicht notwendig.

In [47]:
sql = "SELECT wiagid FROM wiag.monastery"
df_mon_id = Wds.sql_df(sql);

In [48]:
mon_ids_new = setdiff(skipmissing(df_oc.id_monastery), df_mon_id.wiagid)

Int32[]

*2022-02-17* 13 neue Klöster (Historie)

In [88]:
mon_ids_par = join(string.(mon_ids_new), ",")

"3145,3318,3242,3243,3205,2093,40237,30198,60244,3326,4805,30322,30346"

In [91]:
sql = "SELECT * FROM wiag2.institution
WHERE id_gsn IN ($(mon_ids_par))"
df_mon = Wds.sql_df(sql)

,id,comment,name
,Int32,String?,String?
1,80282,missing,"Kollegiatstift St. Margarethe, Ardagger, Österreich"
2,80363,missing,"Kollegiatstift St. Maria, Hofgeismar (zeitweilig in Gottsbüren und Grebenstein)"
3,80435,missing,"Kollegiatstift St. Maria, Lich"
4,80460,missing,"Kollegiatstift St. Johannes Baptista, Mainz"
5,80461,missing,"Kollegiatstift St. Gangolf, Mainz"
6,80513,missing,"Kollegiatstift St. Katharina, Oppenheim"
7,80518,missing,"Kollegiatstift St. Martin, Ober-Mockstadt"
8,81510,missing,Kollegiatstift Hünfeld
9,81570,missing,Benediktinerkloster Seligenstadt


In [94]:
columns = [
    :id => :id_monastery,
    :id_gsn => :wiagid,
    :name => :monastery_name,
]

3-element Vector{Pair{Symbol, Symbol}}:
     :id => :id_monastery
 :id_gsn => :wiagid
   :name => :monastery_name

In [95]:
Wds.filltable!("wiag.monastery", select(df_mon, columns), clear_table = false);

┌ Info: Rows inserted: 13
└ @ WiagDataSetup C:\Users\georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:1206


*2021-12-12* (Historie)

Domstift Verdun (4066) ist in der Klosterdatenbank (siehe unten)

In [58]:
d_mg_mon = Dict(
    4068 => ("Domstift Kurland, Livland", missing),
    4074 => ("Domstift Reval, Estland", "Reval"),
    4075 => ("Domstift Riga, Lettland", "Riga")
);

In [59]:
df_inst = filter(:institution => !ismissing, df_oc);

In [60]:
size(df_inst)

(0, 20)

*2021-12-16* Info (Historie)

In [ ]:
for i_mon in keys(d_mg_mon)
    row_idx = isequal.(df_oc.id_monastery, i_mon)
    df_oc[row_idx, :institution] .= d_mg_mon[i_mon][1]
    df_oc[row_idx, :location] .= d_mg_mon[i_mon][2]
end

In [62]:
df_inst = filter(:institution => !ismissing, df_oc);
df_inst[:, [:id_canon, :office_name, :institution, :location]]

,id_canon,office_name,institution,location
,Int32,SubStri…,Missing,Missing


In [179]:
table_name = "cn_office"
Wds.filltable!(table_name, df_oc, clear_table = true)

┌ Info: Rows inserted: 5679
└ @ WiagDataSetup C:\Users\georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:1202


5679

## Klöster
Aus der Klosterdatenbank beziehen, aufgrund der Ämter aller Domherren  
https://api.gs.sub.uni-goettingen.de/v1/monastery/2077

Zu befüllen: `monastery` und `monastery_location`

*2022-02-18* Siehe unten: Klöster direkt aus dem Dump

In [64]:
using HTTP, JSON, DataFrames, MySQL

In [14]:
url_gs_mon = "https://api.gs.sub.uni-goettingen.de/v1/monastery/"

"https://api.gs.sub.uni-goettingen.de/v1/monastery/"

In [20]:
StringMayBe = Vector{Union{String, Missing}}
df_mon = DataFrame(
    wiagid = Int[],
    name = StringMayBe(),
    patrocinium = StringMayBe()
)
df_mon_loc = DataFrame(
    wiagid_monastery = Int[],
    longitude = StringMayBe(),
    latitude = StringMayBe(),
    placeName = StringMayBe(),
    locationName = StringMayBe(),
    beginTPQ = StringMayBe(),
    endTPQ = StringMayBe()
);

IDs relevanter Klöster lesen

In [13]:
sql = "SELECT DISTINCT id_monastery FROM 
(SELECT id_monastery FROM wiag.office WHERE id_monastery IS NOT NULL
UNION SELECT id_monastery FROM wiag_in.cn_office WHERE id_monastery IS NOT NULL
UNION SELECT id_monastery FROM wiag_in.cn_office_gs WHERE id_monastery IS NOT NULL) AS c"
df_id_mon = Wds.sql_df(sql);

Info

In [83]:
size(df_id_mon)

(395, 1)

In [15]:
in_epic_mon(id) = id in (4623, 20165, 791)
findall(in_epic_mon, df_id_mon.id_monastery)

3-element Vector{Int64}:
 2
 6
 7

Lies Klöster aus der Klosterdatenbank. Wenn ein Kloster nicht gefunden wird, wird ein Dummy-Eintrag angelegt, um einen Absturz der Web-Anwendung zu vermeiden.

In [21]:
id_all = df_id_mon.id_monastery;

In [23]:
id_all[[2, 6, 7]]

3-element Vector{Union{Missing, Int32}}:
   791
  4623
 20165

In [24]:
Wds.get_gs_monasteries!(df_mon, df_mon_loc, id_all)

┌ Info: 80
└ @ WiagDataSetup C:\Users\georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:1343
┌ Info: 160
└ @ WiagDataSetup C:\Users\georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:1343
┌ Warning: Could not parse  (id: 4068)
└ @ WiagDataSetup C:\Users\georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:1373
┌ Warning: Could not parse  (id: 4074)
└ @ WiagDataSetup C:\Users\georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:1373
┌ Warning: Could not parse  (id: 4075)
└ @ WiagDataSetup C:\Users\georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:1373
┌ Info: 240
└ @ WiagDataSetup C:\Users\georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:1343
┌ Info: 320
└ @ WiagDataSetup C:\Users\georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:1343
┌ Warning: Could not parse  (id: 7203)
└ @ WiagDataSetup C:\Users\georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:1373


Info

In [72]:
size(df_mon)

(395, 3)

In [99]:
df_mon[1:7, :]

,wiagid,name
,Int64,String?
1,415,"Benediktinerabtei Corvey, kurzzeitig Domstift"
2,415,"Benediktinerabtei Corvey, kurzzeitig Domstift"
3,791,"Benediktinerkloster St. Michael, Lüneburg"
4,3294,"Benediktinerkloster, dann Kollegiatstift St. Leodegar, Murbach, Frankreich"
5,3428,"Stift Haug, Würzburg"
6,3560,"Kollegiatstift St. Pauli, Bremen, später Benediktinerkloster"
7,4623,"Benediktinerreichsabtei Stablo-Malmedy, Belgien"


In [105]:
df_mon[!, :id_monastery] .= 1:nrow(df_mon);

In [99]:
names(df_mon)

4-element Vector{String}:
 "wiagid"
 "name"
 "patrocinium"
 "id_monastery"

In [106]:
rename!(df_mon, :name => :monastery_name);

In [109]:
table_name = "monastery"
Wds.filltable!(table_name, df_mon, clear_table = true)

┌ Info: Rows inserted: 395
└ @ WiagDataSetup C:\Users\georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:1202


395

In [110]:
df_mon_loc[100:105, :]

,wiagid_monastery,longitude,latitude,placeName,locationName
,Int64,String?,String?,String?,String?
1,982,8.108056,51.571528,Soest,missing
2,983,8.10516666666667,51.5708333333333,Soest,missing
3,1015,10.573333333333,49.302777777778,Ansbach,missing
4,1016,9.146389,49.973611,Aschaffenburg,missing
5,1043,9.1744,47.6625,Konstanz,missing
6,1051,7.3012,50.18369,Treis-Karden,Karden an der Mosel


In [117]:
df_mg_geo = filter(:longitude => ismissing, df_mon_loc);
size(df_mg_geo)

(0, 7)

In [111]:
parseFloat(x) = parse(Float64, x)
transform!(df_mon_loc, :longitude => ByRow(parseFloat) => :longitude);

In [112]:
transform!(df_mon_loc, :latitude => ByRow(parseFloat) => :latitude);

In [122]:
names(df_mon_loc)

7-element Vector{String}:
 "wiagid_monastery"
 "longitude"
 "latitude"
 "placeName"
 "locationName"
 "beginTPQ"
 "endTPQ"

In [124]:
df_mg_place_name = filter(:placeName => ismissing, df_mon_loc);
size(df_mg_place_name)

(0, 7)

In [113]:
df_mon_loc[!, :id] .= 1:nrow(df_mon_loc);

In [114]:
columns = [
    :id => :id,
    :wiagid_monastery => :wiagid_monastery,
    :longitude => :longitude,
    :latitude => :latitude,
    :placeName => :place_name,
    :locationName => :location_name,
    :beginTPQ => :location_begin_tpq,
    :endTPQ => :location_end_tpq
];

In [115]:
table_name = "monastery_location"
Wds.filltable!(table_name, select(df_mon_loc, columns), clear_table = true)

┌ Info: Rows inserted: 422
└ @ WiagDataSetup C:\Users\georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:1202


422

## Klöster (Dump)
Daten aus dem Dump der Klosterdatenbank
https://owncloud.gwdg.de/index.php/s/s4x0TotkZq4Nm2B

Zu befüllen: `monastery` und `monastery_location` und `place`

In [21]:
table_name = "gs_klosterdatenbank.gs_places";
sql = "SELECT * FROM $(table_name)";
df_place = Wds.sql_df(sql);

In [22]:
columns = [
    :id_places,
    :place_name,
    :gemeinde,
    :diocese_id,
    :geonames_id,
    :latitude,
    :longitude,    
]

7-element Vector{Symbol}:
 :id_places
 :place_name
 :gemeinde
 :diocese_id
 :geonames_id
 :latitude
 :longitude

In [23]:
table_name = "place"
Wds.filltable!(table_name, select(df_place, columns))

┌ Info: 10000
└ @ WiagDataSetup C:\Users\georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:1183
┌ Info: 20000
└ @ WiagDataSetup C:\Users\georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:1183
┌ Info: 30000
└ @ WiagDataSetup C:\Users\georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:1183
┌ Info: 40000
└ @ WiagDataSetup C:\Users\georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:1183
┌ Info: Rows inserted: 46493
└ @ WiagDataSetup C:\Users\georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:1206


46493

In [11]:
table_name = "gs_klosterdatenbank.gs_monastery"
sql = "SELECT * FROM $(table_name)"
df_mon = Wds.sql_df(sql);

In [12]:
Wds.clean_up!(df_mon);

In [13]:
size(df_mon)

(5993, 15)

In [14]:
rename!(df_mon, [:id_gsn => :wiagid]);

In [15]:
columns = [
    :id_monastery,
    :wiagid,
    :date_created,
    :created_by_user,
    :status,
    :monastery_name,
    :patrocinium,
    :gs_persons,
    :selection_criteria,
    :last_change,
    :changed_by_user,
    :founder
];

In [16]:
table_name = "monastery"
Wds.filltable!(table_name, select(df_mon, columns), clear_table = true)

┌ Info: Rows inserted: 5993
└ @ WiagDataSetup C:\Users\georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:1206


5993

In [17]:
sql = "SELECT * FROM gs_klosterdatenbank.gs_monastery_location"
df_mon_loc = Wds.sql_df(sql);

In [18]:
names(df_mon_loc)

16-element Vector{String}:
 "id_monastery_location"
 "place_id"
 "relocated"
 "gsn_id"
 "location_begin_tpq"
 "location_begin_taq"
 "location_begin_note"
 "location_end_tpq"
 "location_end_taq"
 "location_end_note"
 "comment"
 "longitude"
 "latitude"
 "location_name"
 "main_location"
 "diocese_id"

In [20]:
df_mon_loc[100:107, [:id_monastery_location, :relocated, :location_name, :place_id]]

,id_monastery_location,relocated,location_name,place_id
,Int32,Int8?,String?,Int32?
1,213,1,missing,17595
2,214,0,missing,16550
3,216,0,missing,17521
4,218,0,missing,36926
5,221,0,missing,16159
6,225,0,Wismar,13796
7,226,0,missing,9583
8,227,0,missing,14671


In [24]:
size(df_mon_loc)

(6452, 16)

In [27]:
dropmissing!(df_mon_loc, :place_id);

In [28]:
size(df_mon_loc)

(6451, 16)

In [30]:
df_mon_loc_db = innerjoin(df_mon_loc, select(df_place, [:id_places, :place_name]), on = :place_id => :id_places);

In [31]:
names(df_mon_loc_db)

17-element Vector{String}:
 "id_monastery_location"
 "place_id"
 "relocated"
 "gsn_id"
 "location_begin_tpq"
 "location_begin_taq"
 "location_begin_note"
 "location_end_tpq"
 "location_end_taq"
 "location_end_note"
 "comment"
 "longitude"
 "latitude"
 "location_name"
 "main_location"
 "diocese_id"
 "place_name"

In [ ]:
rename!(df_mon_loc_db, [:id_monastery_location => :id, :gsn_id => :wiagid_monastery]);

In [41]:
table_name = "monastery_location"
Wds.filltable!(table_name, select(df_mon_loc_db, Not(:comment)), clear_table = true)

6451

┌ Info: Rows inserted: 6451
└ @ WiagDataSetup C:\Users\georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:1206


### Normierte Ämter
`cn_office_norm`  
Die Daten sind lediglich zu übernehmen

In [100]:
sql = "SELECT id_amt_norm as id, AmtNorm as office_name, Bemerkung as comment
FROM domherr.tbl_aemterliste_normiert"
df_occ_nrm = Wds.sql_df(sql);

In [101]:
Wds.clean_up!(df_occ_nrm);

In [103]:
size(df_occ_nrm)

(152, 3)

In [104]:
table_name = "cn_office_norm"
Wds.filltable!(table_name, df_occ_nrm, clear_table = true)

┌ Info: Rows inserted: 152
└ @ WiagDataSetup C:\Users\georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:1206


152

## Hilfstabellen
### `cn_online`
``` sql
CREATE TABLE cn_online LIKE wiag.cn_online;
```

Verknüpfe die Quellen
1. ergänze gsn_id in dh aus ep über id_ep
2. verknüpfe Domherren GS 
3. ergänze id_ep_ep aus ep über gsn_id

Domherren aus der Domherrendatenbank

In [52]:
sql = "SELECT id as id_dh, wiag_episc_id as wiag_episc_id, gsn_id, 
givenname, familyname
FROM wiag_in.cn_canon WHERE status = 'online'"
df_dh_pub = Wds.sql_df(sql);

Extrahiere `id_ep` aus der `wiag_episc_id`

In [53]:
rgx_episc = r"EPISCGatz-([0-9]+)-"
function id_ep(wiagid)
    if ismissing(wiagid)
        return missing
    end
    rgm = match(rgx_episc, wiagid)
    if !isnothing(rgm)
        id_ep = parse(Int, rgm[1])
        return id_ep
    else
        @warn "invalid wiagid " wiagid
        return missing
    end
end

id_ep (generic function with 1 method)

In [54]:
transform!(df_dh_pub, :wiag_episc_id => ByRow(id_ep) => :id_ep);

Bischöfe mit Verweis auf das Digitale Personenregister

In [55]:
sql = "SELECT wiagid as id_ep, gsid as gsn_id_ep
FROM wiag.person
WHERE gsid IS NOT NULL AND gsid <> ''"
df_ep = Wds.sql_df(sql);

In [56]:
parse_int(x) = parse(Int, x)
transform!(df_ep, :id_ep => ByRow(parse_int) => :id_ep);

In [57]:
df_dh_ep = leftjoin(df_dh_pub, df_ep, on = :id_ep, matchmissing = :notequal);

In [58]:
size(df_dh_pub, 1), size(df_dh_ep, 1), count(!ismissing, df_dh_ep.gsn_id_ep), count(ismissing, df_dh_ep.gsn_id)

(2587, 2587, 102, 2165)

In [59]:
df_brunn = subset(df_dh_pub, :id_dh => ByRow(isequal(13324)))

,id_dh,wiag_episc_id,gsn_id,givenname,familyname,id_ep
,Int32,String?,String?,String?,String?,Int64?
1,13324,WIAG-Pers-EPISCGatz-03761-001,missing,Johann,Brunn,3761


In [296]:
df_brunn_ep = subset(df_dh_ep, :id_ep => ByRow(isequal(3761)))
df_brunn_ep[1, [:id_dh, :id_gs, :id_ep, :gsn_id, :gsn_id_ep, :familyname]]

,id_dh,id_gs,id_ep,gsn_id,gsn_id_ep,familyname
,Int32,String?,Int64?,String?,String?,String?
1,13324,missing,3761,missing,059-00936-001,Brunn


In [60]:
update_col(from, to) = ismissing(to) ? from : to;

In [61]:
transform!(df_dh_ep, [:gsn_id_ep, :gsn_id] => ByRow(update_col) => :gsn_id);

In [348]:
count(ismissing, df_dh_ep.gsn_id)

2110

In [309]:
df_brunn_ep = subset(df_dh_ep, :id_ep => ByRow(isequal(3761)))
df_brunn_ep[1, [:id_dh, :id_gs, :id_ep, :gsn_id, :gsn_id_ep, :familyname]]

,id_dh,id_gs,id_ep,gsn_id,gsn_id_ep,familyname
,Int32,String?,Int64?,String?,String?,String?
1,13324,missing,3761,059-00936-001,059-00936-001,Brunn


Domherren aus dem Digitalen Personenregister

In [62]:
sql = "SELECT NULL as id_dh, id as id_gs, gsn_id, 
givenname, familyname,
NULL as wiag_episc_id, NULL as id_ep
FROM wiag_in.cn_canon_gs WHERE status = 'online'"
df_gs = Wds.sql_df(sql);

In [63]:
df_gs_gs = antijoin(df_gs, df_dh_ep, on = :gsn_id, matchmissing = :notequal);

In [64]:
df_dh_gs = vcat(select(df_dh_ep, Not(:gsn_id_ep)), select(df_gs_gs, Not(:id_gs)));

In [65]:
size(df_dh_ep, 1), size(df_gs, 1), size(df_dh_gs, 1)

(2587, 8564, 10790)

Ergänze `id_gs` 

In [66]:
df_dh_gs = leftjoin(df_dh_gs, select(df_gs, [:gsn_id, :id_gs]), on = :gsn_id, matchmissing = :notequal);

Ergänze `id_ep` für Domherren aus dem Digitalen Personenregister

In [67]:
sql = "SELECT wiagid as id_ep_ep, gsid as gsn_id
FROM wiag.person
WHERE gsid IS NOT NULL AND gsid <> ''"
df_ep = Wds.sql_df(sql);

In [68]:
parse_int(x) = parse(Int, x)
transform!(df_ep, :id_ep_ep => ByRow(parse_int) => :id_ep_ep);

In [69]:
df_dh_gs = leftjoin(df_dh_gs, df_ep, on = :gsn_id, matchmissing = :notequal);

In [70]:
count(!ismissing, df_dh_gs.id_ep_ep)

620

In [71]:
transform!(df_dh_gs, [:id_ep_ep, :id_ep] => ByRow(update_col) => :id_ep);

Tests

In [199]:
names(df_dh_gs)

9-element Vector{String}:
 "id_dh"
 "id_gs"
 "wiag_episc_id"
 "gsn_id"
 "givenname"
 "familyname"
 "id_ep"
 "gsn_id_ep"
 "id_ep_ep"

In [326]:
mismatch(x, y) = !ismissing(x) && !ismissing(y) && x != y

mismatch (generic function with 1 method)

In [328]:
df_test_ep = subset(df_dh_gs, [:id_ep, :id_ep_ep] => ByRow(mismatch))

,id_dh,id_gs,wiag_episc_id,gsn_id,givenname,familyname,id_ep,id_ep_ep
,Any,Any,String?,String?,String?,String?,Int64?,Int64?


Setze `wiagid`

In [72]:
function make_wiagid(dst, id)
    if ismissing(id)
        return dst
    else
        return "WIAG-Pers-CANON-" * lpad(id, 5, "0") * "-001"
    end
end
    

make_wiagid (generic function with 1 method)

In [73]:
make_wiagid(missing, 5)

"WIAG-Pers-CANON-00005-001"

In [74]:
df_dh_gs[!, :wiagid] .= missing;

In [75]:
transform!(df_dh_gs, [:wiagid, :id_gs] => ByRow(make_wiagid) => :wiagid);

In [76]:
transform!(df_dh_gs, [:wiagid, :id_dh] => ByRow(make_wiagid) => :wiagid);

In [77]:
function make_wiagid(dst, id)
    if ismissing(id)
        return dst
    else
        return "WIAG-Pers-EPISCGatz-" * lpad(id, 5, "0") * "-001"
    end
end

make_wiagid (generic function with 1 method)

In [78]:
transform!(df_dh_gs, [:wiagid, :id_ep] => ByRow(make_wiagid) => :wiagid);

In [83]:
columns = [:id_dh, :id_gs, :id_ep, :wiagid, :givenname, :familyname]

6-element Vector{Symbol}:
 :id_dh
 :id_gs
 :id_ep
 :wiagid
 :givenname
 :familyname

In [84]:
table_name = "cn_online"
Wds.filltable!(table_name, select(df_dh_gs, columns), clear_table = true)

┌ Info: 10000
└ @ WiagDataSetup C:\Users\georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:1179
┌ Info: Rows inserted: 10790
└ @ WiagDataSetup C:\Users\georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:1202


10790

*2022-02-21*

SQL
``` sql
select count(*) from cn_online;
```
10595

### `cn_namelookup`
``` sql
CREATE TABLE cn_namelookup LIKE wiag.cn_namelookup;
```

In [85]:
Wds.fillcnnamelookup("cn_namelookup", "cn_online", "cn_canon", "cn_canon_gs")

┌ Info: clear 
│   tbllookup = cn_namelookup
└ @ WiagDataSetup C:\Users\georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:691
┌ Info: 400
└ @ WiagDataSetup C:\Users\georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:753
┌ Info: 800
└ @ WiagDataSetup C:\Users\georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:753
┌ Info: 1200
└ @ WiagDataSetup C:\Users\georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:753
┌ Info: 1600
└ @ WiagDataSetup C:\Users\georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:753
┌ Info: 2000
└ @ WiagDataSetup C:\Users\georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:753
┌ Info: 2400
└ @ WiagDataSetup C:\Users\georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:753
┌ Info: 2800
└ @ WiagDataSetup C:\Users\georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:753
┌ Info: 3200
└ @ WiagDataSetup C:\Users\georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:753
┌ Info:

10790

### `cn_office.location_show`

In [195]:
Wds.fillofficelocation("cn_office", "monastery_location", "wiag.place", "cn_online", "id_dh")

┌ Info: 1000
└ @ WiagDataSetup C:\Users\georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:565
┌ Info: 2000
└ @ WiagDataSetup C:\Users\georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:565
┌ Info: 3000
└ @ WiagDataSetup C:\Users\georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:565
┌ Info: 4000
└ @ WiagDataSetup C:\Users\georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:565
┌ Info: 5000
└ @ WiagDataSetup C:\Users\georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:565


5662

In [196]:
Wds.fillofficelocation("cn_office_gs", "monastery_location", "wiag.place", "cn_online", "id_gs")

┌ Info: 1000
└ @ WiagDataSetup C:\Users\georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:565
┌ Info: 2000
└ @ WiagDataSetup C:\Users\georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:565
┌ Info: 3000
└ @ WiagDataSetup C:\Users\georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:565
┌ Info: 4000
└ @ WiagDataSetup C:\Users\georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:565
┌ Info: 5000
└ @ WiagDataSetup C:\Users\georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:565
┌ Info: 6000
└ @ WiagDataSetup C:\Users\georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:565
┌ Info: 7000
└ @ WiagDataSetup C:\Users\georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:565
┌ Info: 8000
└ @ WiagDataSetup C:\Users\georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:565
┌ Info: 9000
└ @ WiagDataSetup C:\Users\georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:565
┌ Info: 10000
└ @ WiagDataSetup C:\Us

14969

4 Orte können nicht bestimmt werden, weil die Amtszeit nicht zu einem angegebenen Zeitraum für das Kloster passt. (2021-11-23)

### `cn_officelookup`

Daten aus der Domherrendatenbank

`cn_office.id_monastery` bezieht sich auf `monastery.wiagid` nicht mehr auf `monastery.id_monastery`. `monastery.wiagid` ist gleichzeitig die Nummer in der Klosterdatenbank.

In [11]:
sql = "SELECT co.id AS id_online, id_office_norm, office_name, " *
"location_show AS location_name, id_monastery, archdeacon_territory, " *
"o.numdate_start AS numdate_start, o.numdate_end AS numdate_end, " *
"o.time_sortkey AS time_sortkey, " *
"ds.name as domstift " *
"FROM cn_office AS o " * 
"JOIN cn_online AS co ON o.id_canon = co.id_dh " *
"LEFT JOIN wiag.domstift AS ds ON o.id_monastery = ds.gs_id"
df_ol_dh = Wds.sql_df(sql);

In [12]:
Wds.clean_up!(df_ol_dh)

In [13]:
size(df_ol_dh)

(5679, 10)

In [14]:
table_name = "cn_officelookup"
Wds.filltable!(table_name, select(df_ol_dh, Not(:id_office_norm)), clear_table = true)

┌ Info: Rows inserted: 5679
└ @ WiagDataSetup C:\Users\georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:1202


5679

Namen der normierten Ämter

In [15]:
sql = "SELECT id, office_name as office_name_norm FROM cn_office_norm"
df_occ_norm = Wds.sql_df(sql);

In [16]:
df_ol_dh_1 = filter(:id_office_norm => !ismissing, df_ol_dh)
df_ol_dh_2 = leftjoin(df_ol_dh_1, df_occ_norm, on = :id_office_norm => :id);

In [17]:
df_ol_dh_on = filter([:office_name, :office_name_norm] => !isequal, df_ol_dh_2);

In [18]:
rename!(df_ol_dh_on, [:office_name => :office_name_orig, :office_name_norm => :office_name]);

In [19]:
unique(df_ol_dh_on[:, [:office_name, :office_name_orig]])[8:15, :]

,office_name,office_name_orig
,String?,String?
1,"Propst, Anwärter",missing
2,"Domkantor, Anwärter",missing
3,"Dompropst, Anwärter",missing
4,Domscholaster,missing
5,"Kanoniker, Anwärter",missing
6,Domdekan,Domdechant
7,Domkellner,Domkellerar
8,Archidiakon,Archdiakon Elze


In [20]:
table_name = "cn_officelookup"
Wds.filltable!(table_name, select(df_ol_dh_on, Not([:id_office_norm, :office_name_orig])), clear_table = false)

┌ Info: Rows inserted: 477
└ @ WiagDataSetup C:\Users\georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:1202


477

Domherren aus dem Digitalen Personenregister

In [21]:
sql = "SELECT co.id AS id_online, office_name, " *
"location_show AS location_name, id_monastery, archdeacon_territory, " *
"o.numdate_start AS numdate_start, o.numdate_end AS numdate_end, " *
"o.time_sortkey AS time_sortkey, " *
"ds.name as domstift " *
"FROM cn_office_gs AS o " * 
"JOIN cn_online AS co ON o.id_canon = co.id_gs " *
"LEFT JOIN wiag.domstift AS ds ON o.id_monastery = ds.gs_id"
df_ol_gs = Wds.sql_df(sql);

In [22]:
Wds.clean_up!(df_ol_gs)

Info

In [192]:
df_ol_gs_ds = filter(:domstift => !ismissing, df_ol_gs);

In [193]:
df_ol_gs_ds[20:26, [:id_online, :office_name, :domstift]]

,id_online,office_name,domstift
,Int32,String,String?
1,168004,Domherr,Paderborn
2,164434,Domherr,Havelberg
3,164412,Domherr,Havelberg
4,168002,Domherr,Paderborn
5,165977,Domherr,Osnabrück
6,164265,Domherr,Osnabrück
7,165933,Domherr,Osnabrück


In [23]:
table_name = "cn_officelookup"
Wds.filltable!(table_name, df_ol_gs, clear_table = false)

┌ Info: 10000
└ @ WiagDataSetup C:\Users\georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:1179
┌ Info: Rows inserted: 14991
└ @ WiagDataSetup C:\Users\georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:1202


14991

### `cn_era`
Struktur: `id_online, domstift, era_start, era_end`  
Es gibt globale Einträge mit `domstift = 'all'` und Einträge pro Domstift. Diese fassen jeweils den Zeitraum pro Domstift zusammen und schließen Einträge ohne Datumsangabe aus. In Doctrine lässt sich das nicht formulieren, da keine Tabelle verknüpft werden kann (join), die aus einer eigenen Abfrage mit Gruppierung entsteht.

global

Personendatenbank

In [128]:
sql = "SELECT co.id AS id_online, co.id_gs, date_hist_first AS era_start, date_hist_last AS era_end, " *
"numdate_birth AS numdate_birth_gs, numdate_death AS numdate_death_gs " *
"FROM cn_online AS co JOIN cn_canon_gs AS c ON co.id_gs = c.id " *
"WHERE status = 'online'"
df_era_gs = Wds.sql_df(sql);

Amtsdaten

In [129]:
sql = "SELECT id_canon as id_gs, min(numdate_start) AS numdate_start_gs, max(numdate_end) AS numdate_end_gs " *
"FROM cn_office_gs GROUP BY id_canon"
df_o_gs = Wds.sql_df(sql);

In [130]:
size(df_era_gs), size(df_o_gs)

((8564, 6), (8491, 3))

In [131]:
df_era_gs = leftjoin(df_era_gs, df_o_gs, on = :id_gs);

In [132]:
size(df_era_gs)

(8564, 8)

Domherrendatenbank

In [133]:
sql = "SELECT co.id AS id_online, co.id_dh, date_hist_first, date_hist_last, " *
"numdate_birth AS numdate_birth_dh, numdate_death AS numdate_death_dh " *
"FROM cn_online AS co JOIN cn_canon AS c ON co.id_dh = c.id " *
"WHERE status = 'online'"
df_era_dh = Wds.sql_df(sql);

Amtsdaten

In [134]:
sql = "SELECT id_canon as id_dh, min(numdate_start) AS numdate_start_dh, max(numdate_end) AS numdate_end_dh " *
"FROM cn_office GROUP BY id_canon"
df_o_dh = Wds.sql_df(sql);

In [135]:
size(df_era_dh), size(df_o_dh)

((2587, 6), (2587, 3))

In [136]:
df_era_dh = leftjoin(df_era_dh, df_o_dh, on = :id_dh);

Zusammenführen

In [137]:
df_era = outerjoin(df_era_gs, df_era_dh, on = :id_online);

In [37]:
size(df_era)

(10595, 15)

Finde Minima und Maxima

In [138]:
function valid_max(x, x_cmp...)
    for x_i in x_cmp
        x = ismissing(x_i) ? x : (ismissing(x) ? x_i : max(x, x_i))
    end
    return x
end
function valid_min(x, x_cmp...)
    for x_i in x_cmp
        x = ismissing(x_i) ? x : (ismissing(x) ? x_i : min(x, x_i))
    end
    return x
end

valid_min (generic function with 1 method)

In [139]:
col_min = [:era_start, :numdate_start_gs, :numdate_start_dh, :date_hist_first]
transform!(df_era, col_min => ByRow(valid_min) => :era_start);

In [140]:
col_max = [:era_end, :numdate_end_gs, :numdate_end_dh, :date_hist_last]
transform!(df_era, col_max => ByRow(valid_max) => :era_end);

Info

In [170]:
df_info = filter(:id_online => in(22614:22619), df_era);

In [171]:
df_info[:, [:id_online, :id_dh, :id_gs, :numdate_birth_gs, :numdate_birth_dh, col_min...]]

,id_online,id_dh,id_gs,numdate_birth_gs,numdate_birth_dh,era_start,numdate_start_gs
,Int32,Int32?,Int32?,Int32?,Int32?,Int32?,Int32?


In [141]:
ne_maybe(x, y) = (ismissing(x) || ismissing(y)) ? false : x != y
df_info = filter([:era_start, :numdate_start_dh] => ne_maybe, df_era);

In [173]:
df_info[12:18, [:id_online, :id_dh, :id_gs, col_min...]]

,id_online,id_dh,id_gs,era_start,numdate_start_gs,numdate_start_dh,date_hist_first
,Int32,Int32?,Int32?,Int32?,Int32?,Int32?,Int32?
1,57412,20369,81379,1371,1371,1408,missing
2,57411,10885,81391,1734,1758,1752,1752
3,57469,10887,81397,1714,1759,1732,missing
4,57468,10896,81428,1683,1683,1716,missing
5,57464,10915,81447,1702,1702,1746,missing
6,57461,10934,81572,1732,1755,1755,1755
7,57460,10936,81574,1740,1761,1761,missing


In [142]:
ne_maybe(x, y) = (ismissing(x) || ismissing(y)) ? false : x != y
df_info = filter([:era_end, :numdate_end_gs] => ne_maybe, df_era);

In [175]:
df_info[12:18, [:id_online, :id_dh, :id_gs, col_max...]]

,id_online,id_dh,id_gs,era_end,numdate_end_gs,numdate_end_dh,date_hist_last
,Int32,Int32?,Int32?,Int32?,Int32?,Int32?,Int32?
1,57462,20382,81506,1432,1429,1432,missing
2,57461,10934,81572,1783,1761,1761,1761
3,57455,10944,81633,1825,1802,1789,missing
4,57451,10965,81795,1804,1797,1804,1804
5,57444,11019,81956,1777,1772,1777,missing
6,57442,11025,82013,1775,1734,1775,missing
7,57474,10891,83911,1798,1759,1798,missing


Geburts- und Sterbedatum  
Geburts- und Sterbedaten - so vorhanden - können die Zeitspanne nur verkürzen!

In [143]:
function era_birth(era, b1, b2)
    b = valid_min(b1, b2)
    return ismissing(b) ? era : b
end
function era_death(era, b1, b2)
    b = valid_max(b1, b2)
    return ismissing(b) ? era : b
end
    

era_death (generic function with 1 method)

In [144]:
transform!(df_era, [:era_start, :numdate_birth_dh, :numdate_birth_gs] => ByRow(era_birth) => :era_start);

In [145]:
transform!(df_era, [:era_end, :numdate_death_dh, :numdate_death_gs] => ByRow(era_death) => :era_end);

Info

In [179]:
df_info = filter([:era_start, :numdate_birth_dh] => ne_maybe, df_era);

In [182]:
df_info[!, [:id_online, :id_dh, :id_gs, :numdate_birth_gs, :numdate_birth_dh, col_min...]]

,id_online,id_dh,id_gs,numdate_birth_gs,numdate_birth_dh,era_start,numdate_start_gs
,Int32,Int32?,Int32?,Int32?,Int32?,Int32?,Int32?
1,19007,10919,81359,1738,1742,1738,1756
2,19181,11021,85597,1735,1740,1735,1756
3,19188,10984,85858,1746,1752,1746,1770
4,19225,11054,86880,1750,1756,1750,1760


In [180]:
df_info = filter(:numdate_birth_gs => ismissing, df_era);

In [181]:
df_info[100:105, [:id_online, :id_dh, :id_gs, :numdate_birth_gs, :numdate_birth_dh, col_min...]]

,id_online,id_dh,id_gs,numdate_birth_gs,numdate_birth_dh,era_start,numdate_start_gs
,Int32,Int32?,Int32?,Int32?,Int32?,Int32?,Int32?
1,57554,13639,84325,missing,missing,1201,1201
2,57553,20425,84327,missing,missing,1301,1301
3,57552,26590,84330,missing,missing,1337,1337
4,57551,13714,84332,missing,missing,1237,1240
5,57550,20412,84333,missing,missing,1301,1301
6,57549,20482,84334,missing,missing,1294,1311


In [182]:
df_info = filter([:era_end, :numdate_death_dh] => ne_maybe, df_era);

In [186]:
df_info[:, [:id_online, :id_dh, :id_gs, :numdate_death_gs, :numdate_death_dh, col_max...]]

,id_online,id_dh,id_gs,numdate_death_gs,numdate_death_dh,era_end,numdate_end_gs
,Int32,Int32?,Int32?,Int32?,Int32?,Int32?,Int32?


In [187]:
df_info = filter(:numdate_death_gs => ismissing, df_era);

In [183]:
df_info[100:105, [:id_online, :id_dh, :id_gs, :numdate_death_gs, :numdate_death_dh, col_max...]]

LoadError: BoundsError: attempt to access 0×15 DataFrame at index [100:105, [:id_online, :id_dh, :id_gs, :numdate_death_gs, :numdate_death_dh, :era_end, :numdate_end_gs, :numdate_end_dh, :date_hist_last]]

Ersetze fehlende Werte durch den jeweils anderen Wert, falls vorhanden. (NULL erscheint in Sortierungen immer als erstes)

In [146]:
function replace_missing(a, b)
    if ismissing(a)
        a = b
    elseif ismissing(b)
        b = a
    end
    return a, b
end

replace_missing (generic function with 1 method)

In [147]:
transform!(df_era, [:era_start, :era_end] => ByRow(replace_missing) => [:era_start, :era_end]);

Schreibe Daten für `domstift = 'all'`

In [148]:
df_era[!, :domstift] .= "all";

In [149]:
table_name = "cn_era"
Wds.filltable!(table_name, select(df_era, :id_online, :domstift, :era_start, :era_end), clear_table = true)

┌ Info: 10000
└ @ WiagDataSetup C:\Users\georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:1179
┌ Info: Rows inserted: 10790
└ @ WiagDataSetup C:\Users\georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:1202


10790

Zeitraum pro Domstift

Personendatenbank

In [150]:
sql_union = "SELECT ol_1.id AS id_online, id_canon, d_1.name as domstift, numdate_start, numdate_end " *
"FROM cn_office_gs AS o_gs " *
"JOIN wiag.domstift AS d_1 on o_gs.id_monastery = d_1.gs_id JOIN cn_online AS ol_1 ON ol_1.id_gs = o_gs.id_canon " *
"UNION (SELECT ol.id AS id_online, id_canon, d.name as domstift, numdate_start, numdate_end " *
"FROM cn_office AS o " *
"JOIN wiag.domstift as d on o.id_monastery = d.gs_id JOIN cn_online AS ol ON ol.id_dh = o.id_canon)"
df_od = Wds.sql_df(sql_union);

In [49]:
size(df_od)

(14358, 5)

Info

In [151]:
sql_dh = "SELECT ol.id AS id_online, id_canon, d.name as domstift, numdate_start, numdate_end " *
"FROM cn_office AS o " *
"JOIN wiag.domstift as d on o.id_monastery = d.gs_id JOIN cn_online AS ol ON ol.id_dh = o.id_canon "
df_od_dh = Wds.sql_df(sql_dh);

In [154]:
size(df_od_dh)

(4415, 5)

In [152]:
sql = "SELECT id_online, domstift, min(numdate_start) AS era_start, max(numdate_end) AS era_end " *
"FROM (" * sql_union * ") AS U " *
"GROUP BY id_online, domstift"
df_era_d = Wds.sql_df(sql);

In [153]:
size(df_era_d), length(unique(df_era_d.id_online)), length(unique(df_era_d.domstift))

((12966, 4), 10667, 34)

Info

In [194]:
filter(:id_online => isequal(22501), df_era_d)

,id_online,domstift,era_start,era_end
,Int32,String?,Int32?,Int32?


Ersetze fehlende Werte durch den jeweils anderen Wert, falls vorhanden. (NULL erscheint in Sortierungen immer als erstes)

In [155]:
transform!(df_era_d, [:era_start, :era_end] => ByRow(replace_missing) => [:era_start, :era_end]);

Schreibe Daten für die einzelnen Domstifte

In [156]:
table_name = "cn_era"
Wds.filltable!(table_name, select(df_era_d, :id_online, :domstift, :era_start, :era_end), clear_table = false)

┌ Info: 10000
└ @ WiagDataSetup C:\Users\georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:1179
┌ Info: Rows inserted: 12966
└ @ WiagDataSetup C:\Users\georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:1202


12966

### `cn_idlookup`

Liese relevante Domherren aus `cn_online`

In [157]:
sql = "SELECT id as id_online, id_dh FROM cn_online WHERE id_dh IS NOT NULL"
df_online_dh = Wds.sql_df(sql);

In [158]:
sql = "SELECT * FROM cn_canon"
df_dh = Wds.sql_df(sql);

Externe Identifier für Domherren aus der Domherrendatenbank

In [159]:
df_idol = DataFrame(:id_dh => Int[], :authority_id => String[])

,id_dh,authority_id
,Int64,String


In [160]:
for row in eachrow(df_dh)
    for col in (:gnd_id, :gsn_id, :viaf_id, :wikidata_id, :wiag_episc_id)
        if !ismissing(row[col])
            push!(df_idol, (row[:id], row[col]))
        end
    end
end

In [161]:
size(df_idol)

(1740, 2)

In [162]:
df_idol = leftjoin(df_idol, df_online_dh, on = :id_dh);

In [163]:
table_name = "cn_idlookup"
Wds.filltable!(table_name, select(df_idol, :id_online, :authority_id), clear_table = true)

┌ Info: Rows inserted: 1740
└ @ WiagDataSetup C:\Users\georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:1202


1740

In [164]:
sql = "SELECT id as id_online, id_gs FROM cn_online WHERE id_gs IS NOT NULL"
df_online_gs = Wds.sql_df(sql);

In [165]:
sql = "SELECT * FROM cn_canon_gs WHERE status = 'online'"
df_gs = Wds.sql_df(sql);

In [166]:
size(df_gs)

(8564, 34)

In [170]:
df_idol = DataFrame(:id_gs => Int[], :authority_id => String[])

,id_gs,authority_id
,Int64,String


In [171]:
names(df_idol)

2-element Vector{String}:
 "id_gs"
 "authority_id"

In [172]:
for row in eachrow(df_gs)
    for col in (:gnd_id, :gsn_id, :viaf_id, :wikidata_id, :wiag_episc_id)
        if !ismissing(row[col])
            push!(df_idol, (row[:id], row[col]))
        end
    end
end

In [173]:
size(df_idol)

(9608, 2)

In [174]:
df_idol = leftjoin(df_idol, df_online_gs, on = :id_gs);

In [175]:
size(df_idol)

(9608, 3)

In [176]:
table_name = "cn_idlookup"
Wds.filltable!(table_name, select(df_idol, :id_online, :authority_id), clear_table = false)

┌ Info: Rows inserted: 9608
└ @ WiagDataSetup C:\Users\georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:1202


9608

Daten in die Zieldatenbank übernehmen (im eingespielten Prozess können die Tabellen auch direkt vom Notebook aus beschrieben werden)
```sql
use wiag;
delete from cn_reference;
insert into cn_reference (select * from wiag_in.cn_reference);

delete from cn_canon;
insert into cn_canon (select * from wiag_in.cn_canon);
delete from cn_canon_reference;
insert into cn_canon_reference (select * from wiag_in.cn_canon_reference);
delete from cn_office;
insert into cn_office (select * from wiag_in.cn_office);

delete from cn_online;
insert into cn_online (select * from wiag_in.cn_online);
delete from cn_namelookup;
insert into cn_namelookup (select * from wiag_in.cn_namelookup);

drop table cn_officelookup;
create table cn_officelookup like wiag_in.cn_officelookup;
insert into cn_officelookup (select * from wiag_in.cn_officelookup);

drop table cn_office_norm
create table cn_office_norm like wiag_in.cn_office_norm;
insert into cn_office_norm (select * from wiag_in.cn_office_norm);

drop table cn_era;
create table cn_era like wiag_in.cn_era;
insert into cn_era (select * from wiag_in.cn_era);

drop table cn_idlookup;
create table cn_idlookup like wiag_in.cn_idlookup;
insert into cn_idlookup (select * from wiag_in.cn_idlookup);

drop table monastery;
create table monastery like wiag_in.monastery;
insert into monastery (select * from wiag_in.monastery);

drop table monastery_location;
create table monastery_location like wiag_in.monastery_location;
insert into monastery_location (select * from wiag_in.monastery_location);
```